In [17]:
import time
import oci
import os
from dotenv import load_dotenv
from PyPDF2 import PdfReader
import oracledb
from langchain_community.vectorstores import oraclevs
from langchain_community.vectorstores.oraclevs import OracleVS
from langchain_community.vectorstores.utils import DistanceStrategy
from langchain_core.documents import Document
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.text_splitter import CharacterTextSplitter
# from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores.utils import DistanceStrategy
from langchain_core.documents import Document
from langchain_community.llms import OCIGenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
import requests
from PyPDF2 import PdfReader
from io import BytesIO
import re
from langchain.text_splitter import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer
from langchain_core.embeddings import Embeddings
print("Successfully imported libraries and modules")

Successfully imported libraries and modules


In [2]:
# Function to extract text from a PDF
def extract_text_from_pdf(pdf):
    text = ""
    for page in pdf.pages:
        text += page.extract_text()
    return text

In [3]:
# Function to split text by headings using a regex pattern
def split_by_headings(text):
    heading_pattern = re.compile(r'(?:^|\n)([A-Z][^\n]*)\n')
    sections = re.split(heading_pattern, text)
    chunks = []
    for i in range(0, len(sections), 2):
        if i + 1 < len(sections):
            current_chunk = sections[i].strip() + "\n" + sections[i+1].strip()
        else:
            current_chunk = sections[i].strip()
        chunks.append(current_chunk)
    return chunks

In [4]:
# Function for dynamic chunking based on document length
def dynamic_chunking(text, num_pages):
    # Adjust chunk size based on document length
    if num_pages > 300:
        max_chunk_size = 2000
        overlap = 300
    elif num_pages > 50:
        max_chunk_size = 1000
        overlap = 200
    else:
        max_chunk_size = 500
        overlap = 100
    # Implement a basic chunking logic
    sentences = text.split('.')
    chunks = []
    current_chunk = ""
    for sentence in sentences:
        if len(current_chunk) + len(sentence) <= max_chunk_size:
            current_chunk += sentence + '.'
        else:
            chunks.append(current_chunk.strip())
            current_chunk = sentence[-overlap:] + '.'
    if current_chunk:
        chunks.append(current_chunk.strip())
    return chunks

In [5]:
# Function to perform advanced chunking
def advanced_chunking(text, num_pages):
    sections = split_by_headings(text)
    all_chunks = []
    for section in sections:
        all_chunks.extend(dynamic_chunking(section, num_pages))
    return all_chunks

In [6]:
# Function to convert chunks into Document objects with metadata
def chunks_to_docs(chunks, doc_id):
    docs = []
    for i, chunk in enumerate(chunks):
        metadata = {'id': f"{doc_id}_chunk_{i}", 'page': f"Page {i+1}"}
        docs.append(Document(page_content=chunk, metadata=metadata))
    return docs

In [7]:
def load_pdf_from_url(url):
    """
    Downloads a PDF from the given URL and returns a PdfReader object.
    :param url: The URL of the PDF file.
    :return: PdfReader object.
    """
    response = requests.get(url)
    if response.status_code == 200:
        pdf_file = BytesIO(response.content)
        return PdfReader(pdf_file)
    else:
        print(f"Failed to download PDF from {url}")
        return None


In [8]:
pdf_urls = [
    # Performing Self Service Tasks with Oracle Identity Governance
    "https://docs.oracle.com/en/middleware/idm/identity-governance/12.2.1.4/oigdk/reference-oracle-identity-governance-docker-and-kubernetes.pdf",

    # Administering Oracle Identity Governance
    "https://docs.oracle.com/en/middleware/idm/identity-governance/12.2.1.4/omadm/administering-oracle-identity-governance.pdf",

    # Developing and Customizing Applications for Oracle Identity Governance
    "https://docs.oracle.com/en/middleware/idm/identity-governance/12.2.1.4/omdev/developing-and-customizing-applications-oracle-identity-governance.pdf",

    # Release Notes for Oracle Identity Management
    "https://docs.oracle.com/en/middleware/idm/suite/12.2.1.4/idmrn/release-notes-oracle-identity-management.pdf",

    # Upgrading Oracle Identity Manager
    "https://docs.oracle.com/en/middleware/fusion-middleware/12.2.1.4/iamup/upgrading-oracle-identity-manager.pdf",

    # Performing Self Service Tasks with Oracle Identity Governance
    "https://docs.oracle.com/en/middleware/idm/identity-governance/12.2.1.4/omusg/performing-self-service-tasks-oracle-identity-governance.pdf",

    # Help Topics for Oracle Identity Governance
    "https://docs.oracle.com/en/middleware/idm/identity-governance/12.2.1.4/omhlp/help-topics-oracle-identity-governance.pdf"

]

pdf_readers = [load_pdf_from_url(url) for url in pdf_urls]

# Check if all PDFs were loaded successfully
for i, pdf in enumerate(pdf_readers):
    if pdf:
        print(f"PDF {i+1} loaded successfully with {len(pdf.pages)} pages.")
    else:
        print(f"PDF {i+1} failed to load.")



PDF 1 loaded successfully with 5 pages.
PDF 2 loaded successfully with 719 pages.
PDF 3 loaded successfully with 810 pages.
PDF 4 loaded successfully with 83 pages.
PDF 5 loaded successfully with 281 pages.
PDF 6 loaded successfully with 486 pages.
PDF 7 loaded successfully with 325 pages.


In [10]:
pdf_folder = "PDFs"
if not os.path.exists(pdf_folder):
    os.makedirs(pdf_folder)

# List of URLs to download PDFs from
pdf_urls = [
    "https://docs.oracle.com/en/middleware/idm/identity-governance/12.2.1.4/oigdk/reference-oracle-identity-governance-docker-and-kubernetes.pdf",
    "https://docs.oracle.com/en/middleware/idm/identity-governance/12.2.1.4/omadm/administering-oracle-identity-governance.pdf",
    "https://docs.oracle.com/en/middleware/idm/identity-governance/12.2.1.4/omdev/developing-and-customizing-applications-oracle-identity-governance.pdf",
    "https://docs.oracle.com/en/middleware/idm/suite/12.2.1.4/idmrn/release-notes-oracle-identity-management.pdf",
    "https://docs.oracle.com/en/middleware/fusion-middleware/12.2.1.4/iamup/upgrading-oracle-identity-manager.pdf",
    "https://docs.oracle.com/en/middleware/idm/identity-governance/12.2.1.4/omusg/performing-self-service-tasks-oracle-identity-governance.pdf",
    "https://docs.oracle.com/en/middleware/idm/identity-governance/12.2.1.4/omhlp/help-topics-oracle-identity-governance.pdf"
]

# Function to download a PDF file
def download_pdf(url, folder):
    response = requests.get(url)
    if response.status_code == 200:
        pdf_name = url.split("/")[-1]
        pdf_path = os.path.join(folder, pdf_name)
        with open(pdf_path, 'wb') as pdf_file:
            pdf_file.write(response.content)
        print(f"{pdf_name} downloaded successfully.")
    else:
        print(f"Failed to download {url}")

# Download all PDFs
for url in pdf_urls:
    download_pdf(url, pdf_folder)

reference-oracle-identity-governance-docker-and-kubernetes.pdf downloaded successfully.
administering-oracle-identity-governance.pdf downloaded successfully.
developing-and-customizing-applications-oracle-identity-governance.pdf downloaded successfully.
release-notes-oracle-identity-management.pdf downloaded successfully.
upgrading-oracle-identity-manager.pdf downloaded successfully.
performing-self-service-tasks-oracle-identity-governance.pdf downloaded successfully.
help-topics-oracle-identity-governance.pdf downloaded successfully.


In [11]:
load_dotenv()

username = os.getenv("DB_USERNAME")
password = os.getenv("DB_PASSWORD")
dsn = os.getenv("DB_DSN")
COMPARTMENT_OCID = os.getenv("COMPARTMENT_OCID")

print("The database user name is:", username)

try: 
    conn23c = oracledb.connect(user=username, password=password, dsn=dsn)
    print("Connection successful!")
except Exception as e:
    print("Connection failed!")


The database user name is: admin
Connection successful!


In [12]:
# Initialize the embedding model
model_4db = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

/home/opc/.conda/envs/oig/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [15]:
for i, pdf in enumerate(pdf_readers):
    if pdf:
        try:
            print("There is a PDF")
            # Step 1: Extract text from the PDF
            pdf_text = extract_text_from_pdf(pdf)
            num_pages = len(pdf.pages)
            print("Step 1 Complete")
            # Step 2 & 3: Apply advanced chunking
            chunks = advanced_chunking(pdf_text, num_pages)
            print("Step 2 & 3 Complete")
            # Step 4: Convert chunks to Document objects
            doc_id = f"PDF_{i+1}"
            docs = chunks_to_docs(chunks, doc_id)
            print("Step 4 Complete")
            # Step 5: Store chunks in Oracle Vector Store
            texts = [doc.page_content for doc in docs]  # Extract the text content for embedding
            embeddings = model_4db.encode(texts)
            print("Embeddings Complete")
            vector_stores = OracleVS.from_documents(
                docs,
                lambda x: model_4db.encode([x])[0],  # Embed each text chunk
                client=conn23c,
                table_name="OIG_info_table",
                distance_strategy=DistanceStrategy.COSINE
            )
            print(f"Chunks from PDF {i+1} processed and stored successfully.")
        except Exception as e:
            print(f"An error occurred while processing PDF {i+1}: {e}")

There is a PDF
Step 1 Complete
Step 2 & 3 Complete
Step 4 Complete
Embeddings Complete


KeyboardInterrupt: 

In [20]:
# Create a wrapper to make the SentenceTransformer compatible with the expected Langchain embeddings interface
class HuggingFaceEmbeddings(Embeddings):
    def __init__(self, model):
        self.model = model
    def embed_documents(self, documents):
        """Embed a list of documents (text chunks)."""
        return self.model.encode(documents)
    def embed_query(self, query):
        """Embed a single query."""
        return self.model.encode([query])[0]
# Initialize the embedding model
model_4db = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
# Initialize your custom embedding function
hf_embeddings = HuggingFaceEmbeddings(model_4db)

In [21]:
# Process each PDF (remaining part of your code)
for i, pdf in enumerate(pdf_readers):
    if pdf:
        try:
            print("There is a PDF")
            # Step 1: Extract text from the PDF
            pdf_text = extract_text_from_pdf(pdf)
            num_pages = len(pdf.pages)
            print("Step 1 Complete")
            # Step 2 & 3: Apply advanced chunking
            chunks = advanced_chunking(pdf_text, num_pages)
            print("Step 2 & 3 Complete")
            # Step 4: Convert chunks to Document objects
            doc_id = f"PDF_{i+1}"
            docs = chunks_to_docs(chunks, doc_id)
            print("Step 4 Complete")
            # Step 5: Store chunks in Oracle Vector Store using the correct embedding function
            vector_stores = OracleVS.from_documents(
                docs,
                hf_embeddings,  # Use the custom embedding wrapper
                client=conn23c,
                table_name="OIG_info_table",
                distance_strategy=DistanceStrategy.COSINE
            )
            print(f"Chunks from PDF {i+1} processed and stored successfully.")
        except Exception as e:
            print(f"An error occurred while processing PDF {i+1}: {e}")

There is a PDF
Step 1 Complete
Step 2 & 3 Complete
Step 4 Complete


2024-08-14 22:39:35,373 - ERROR - An unexpected error occurred: DPY-4011: the database or network closed the connection
Help: https://python-oracledb.readthedocs.io/en/latest/user_guide/troubleshooting.html#dpy-4011
Traceback (most recent call last):
  File "/home/opc/.conda/envs/oig/lib/python3.9/site-packages/langchain_community/vectorstores/oraclevs.py", line 54, in wrapper
    return func(*args, **kwargs)
  File "/home/opc/.conda/envs/oig/lib/python3.9/site-packages/langchain_community/vectorstores/oraclevs.py", line 427, in drop_table_purge
    if _table_exists(client, table_name):
  File "/home/opc/.conda/envs/oig/lib/python3.9/site-packages/langchain_community/vectorstores/oraclevs.py", line 84, in _table_exists
    cursor.execute(f"SELECT COUNT(*) FROM {table_name}")
  File "/home/opc/.conda/envs/oig/lib/python3.9/site-packages/oracledb-2.4.0b1-py3.9-linux-aarch64.egg/oracledb/cursor.py", line 701, in execute
    impl.execute(self)
  File "src/oracledb/impl/thin/cursor.pyx", li

An error occurred while processing PDF 1: Failed due to a DB issue: Unexpected error: DPY-4011: the database or network closed the connection
Help: https://python-oracledb.readthedocs.io/en/latest/user_guide/troubleshooting.html#dpy-4011
There is a PDF
Step 1 Complete
Step 2 & 3 Complete


2024-08-14 22:39:45,565 - ERROR - An unexpected error occurred: 'NoneType' object has no attribute 'send'
Traceback (most recent call last):
  File "/home/opc/.conda/envs/oig/lib/python3.9/site-packages/langchain_community/vectorstores/oraclevs.py", line 54, in wrapper
    return func(*args, **kwargs)
  File "/home/opc/.conda/envs/oig/lib/python3.9/site-packages/langchain_community/vectorstores/oraclevs.py", line 427, in drop_table_purge
    if _table_exists(client, table_name):
  File "/home/opc/.conda/envs/oig/lib/python3.9/site-packages/langchain_community/vectorstores/oraclevs.py", line 84, in _table_exists
    cursor.execute(f"SELECT COUNT(*) FROM {table_name}")
  File "/home/opc/.conda/envs/oig/lib/python3.9/site-packages/oracledb-2.4.0b1-py3.9-linux-aarch64.egg/oracledb/cursor.py", line 701, in execute
    impl.execute(self)
  File "src/oracledb/impl/thin/cursor.pyx", line 178, in oracledb.thin_impl.ThinCursorImpl.execute
  File "src/oracledb/impl/thin/protocol.pyx", line 440, i

Step 4 Complete
An error occurred while processing PDF 2: Failed due to a DB issue: Unexpected error: 'NoneType' object has no attribute 'send'
There is a PDF


...

Step 1 Complete
Step 2 & 3 Complete
Step 4 Complete
An error occurred while processing PDF 5: Failed due to a DB issue: Unexpected error: 'NoneType' object has no attribute 'send'
There is a PDF


2024-08-14 22:40:08,258 - ERROR - An unexpected error occurred: 'NoneType' object has no attribute 'send'
Traceback (most recent call last):
  File "/home/opc/.conda/envs/oig/lib/python3.9/site-packages/langchain_community/vectorstores/oraclevs.py", line 54, in wrapper
    return func(*args, **kwargs)
  File "/home/opc/.conda/envs/oig/lib/python3.9/site-packages/langchain_community/vectorstores/oraclevs.py", line 427, in drop_table_purge
    if _table_exists(client, table_name):
  File "/home/opc/.conda/envs/oig/lib/python3.9/site-packages/langchain_community/vectorstores/oraclevs.py", line 84, in _table_exists
    cursor.execute(f"SELECT COUNT(*) FROM {table_name}")
  File "/home/opc/.conda/envs/oig/lib/python3.9/site-packages/oracledb-2.4.0b1-py3.9-linux-aarch64.egg/oracledb/cursor.py", line 701, in execute
    impl.execute(self)
  File "src/oracledb/impl/thin/cursor.pyx", line 178, in oracledb.thin_impl.ThinCursorImpl.execute
  File "src/oracledb/impl/thin/protocol.pyx", line 440, i

Step 1 Complete
Step 2 & 3 Complete
Step 4 Complete
An error occurred while processing PDF 6: Failed due to a DB issue: Unexpected error: 'NoneType' object has no attribute 'send'
There is a PDF


2024-08-14 22:40:12,788 - ERROR - An unexpected error occurred: 'NoneType' object has no attribute 'send'
Traceback (most recent call last):
  File "/home/opc/.conda/envs/oig/lib/python3.9/site-packages/langchain_community/vectorstores/oraclevs.py", line 54, in wrapper
    return func(*args, **kwargs)
  File "/home/opc/.conda/envs/oig/lib/python3.9/site-packages/langchain_community/vectorstores/oraclevs.py", line 427, in drop_table_purge
    if _table_exists(client, table_name):
  File "/home/opc/.conda/envs/oig/lib/python3.9/site-packages/langchain_community/vectorstores/oraclevs.py", line 84, in _table_exists
    cursor.execute(f"SELECT COUNT(*) FROM {table_name}")
  File "/home/opc/.conda/envs/oig/lib/python3.9/site-packages/oracledb-2.4.0b1-py3.9-linux-aarch64.egg/oracledb/cursor.py", line 701, in execute
    impl.execute(self)
  File "src/oracledb/impl/thin/cursor.pyx", line 178, in oracledb.thin_impl.ThinCursorImpl.execute
  File "src/oracledb/impl/thin/protocol.pyx", line 440, i

Step 1 Complete
Step 2 & 3 Complete
Step 4 Complete
An error occurred while processing PDF 7: Failed due to a DB issue: Unexpected error: 'NoneType' object has no attribute 'send'
